<a href="https://colab.research.google.com/github/radiaated/DecisionTreeRegressionScratch/blob/main/ID3Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [394]:
# Import modules
import numpy as np
import pandas as pd

# ID3

In [395]:
# Helper functions
def cov(values: pd.Series):
  """
  Compute the coefficient of variation (CoV) for a numeric pandas Series.

  The coefficient of variation is defined as the ratio of the standard
  deviation to the mean:

      CoV = std(values) / mean(values)

  Parameters
  ----------
  values : pd.Series
      A pandas Series containing numeric values.

  Returns
  -------
  float
      The coefficient of variation of the input values.
  """


  return np.std(values) / np.mean(values)

def stdr_xy(X: pd.Series, y: pd.Series):
  """
  Compute the standard deviation reduction (STDR) of target variable y
  after splitting by the categorical values in X.

  The function calculates:
      STDR = std(y) - sum_over_groups( (n_i / n) * std(y_i) )

  where:
      - y_i is the subset of y corresponding to each unique value in X
      - n_i is the size of each subset
      - n is the total number of samples

  Parameters
  ----------
  X : pd.Series
      A pandas Series containing categorical or discrete grouping values.
  y : pd.Series
      A pandas Series containing numeric target values aligned with X.

  Returns
  -------
  float
      The reduction in standard deviation of y after partitioning by X.
  """

  stdr = 0
  for val in X.unique():

    x_value = X[X == val]
    y_value = y.loc[x_value.index]

    stdr += (len(x_value) / len(X)) * np.std(y_value)

  stdr = np.std(y) - stdr

  return stdr

In [396]:
class DecisionNode():
  """
  A node in a decision tree structure.

  A DecisionNode can represent either:
  - An internal decision node that splits on a feature and contains branches, or
  - A leaf node that stores a predicted value.

  Parameters
  ----------
  feature : str, optional
      The name of the feature used for splitting at this node.
      Should be None for leaf nodes.
  branches : dict, optional
      A dictionary mapping feature values to child DecisionNode objects.
      Used only for internal (non-leaf) nodes.
  leaf_value : float, optional
      The prediction value stored in the node if it is a leaf.
      Should be None for internal nodes.

  Attributes
  ----------
  feature : str
      Feature used for splitting at this node.
  branches : dict
      Mapping of feature values to child nodes.
  leaf_value : float
      Prediction value if the node is a leaf.
  """

  def __init__(self, feature: str = None, branches: dict =None, leaf_value: float =None):

    self.feature = feature
    self.branches = branches
    self.leaf_value = leaf_value

In [397]:
class DecisionTree():
  """
  A simple Decision Tree implementation for regression tasks.

  The tree splits data based on the feature that maximizes a custom
  standard deviation reduction metric (stdr_xy). Splitting stops when:
  - The number of samples is less than or equal to min_samples
  - The maximum depth is reached
  - The covariance of the target values is below the splitting threshold

  Parameters
  ----------
  spitting_threhold : float
      Minimum covariance threshold required to continue splitting.
  max_depth : int
      Maximum depth allowed for the tree.
  min_samples : int
      Minimum number of samples required to split a node.
  """

  def __init__(self, spitting_threhold: float, max_depth: int, min_samples: int):
    """
    Initialize the Decision Tree with stopping criteria.

    Parameters
    ----------
    spitting_threhold : float
        Minimum covariance required to continue splitting.
    max_depth : int
        Maximum depth of the tree.
    min_samples : int
        Minimum number of samples required to split.
    """

    # Threshold for covariance stopping condition
    self.spitting_threhold = spitting_threhold

    # Maximum allowed depth of the tree
    self.max_depth = max_depth

    # Minimum number of samples required to perform a split
    self.min_samples = min_samples

    # Current depth of the tree
    self.depth = 0

    # Root node of the tree (DecisionNode)
    self.root: DecisionNode = None


  def build(self, X_train: pd.DataFrame, y_train: pd.Series):
    """
    Recursively builds the decision tree.

    Parameters
    ----------
    X_train : pd.DataFrame
        Feature dataset.
    y_train : pd.Series
        Target values.

    Returns
    -------
    DecisionNode
        A node representing either a leaf or an internal split node.
    """

    # Stopping conditions:
    # 1. Too few samples
    # 2. Maximum depth reached
    # 3. Target covariance is below threshold
    if len(X_train) <= self.min_samples or self.depth >= self.max_depth or cov(y_train) <= self.spitting_threhold:

      # Create a leaf node with the mean target value
      return DecisionNode(leaf_value = np.mean(y_train).item())

    # Track the best standard deviation reduction
    max_stdr = -1

    # Feature chosen for splitting
    split_feature = None

    # Iterate over all features to find the best split
    for col in X_train.columns:

      # Compute custom standard deviation reduction metric
      stdr = stdr_xy(X_train[col], y_train)

      # Update best feature if improvement found
      if stdr > max_stdr:

        max_stdr = stdr
        split_feature = col

    # Dictionary to store child branches
    branches = {}

    # Create a branch for each unique value of the selected feature
    for col_value in X_train[split_feature].unique():

      # Subset of X where feature equals the specific value
      X_col_value = X_train[X_train[split_feature] == col_value]

      # Corresponding target values
      y_col_value = y_train.loc[X_col_value.index]

      # Remove the splitting feature from the subset
      X_col_value = X_col_value.drop(split_feature, axis=1)

      # Recursively build subtree for this branch
      branches[col_value] = self.build(X_col_value, y_col_value)

    # Increase depth after building this level
    self.depth += 1

    # Return internal decision node
    return DecisionNode(feature = split_feature, branches = branches)


  def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
    """
    Train the Decision Tree on the provided dataset.

    Parameters
    ----------
    X_train : pd.DataFrame
        Training features.
    y_train : pd.Series
        Training target values.
    """

    # Build the tree starting from the root
    self.root = self.build(X_train, y_train)


  def __visualize_node(self, node: DecisionNode, depth = 0):
    """
    Recursively prints a visual representation of the tree.

    Parameters
    ----------
    node : DecisionNode
        Current node being visualized.
    depth : int
        Current depth (used for indentation).
    """

    # If this is a leaf node, print its value
    if node.leaf_value:
      print(node.leaf_value)
      return

    # Print feature used for splitting
    print("[ %s ]" % node.feature)

    # Recursively print branches
    for feature_value, node in node.branches.items():

      # Indentation based on depth level
      print("\t" * depth + " -- " + feature_value + " --> ", end="")

      self.__visualize_node(node, depth = depth + 1)


  def visualize(self):
    """
    Public method to print the full tree structure.
    """

    self.__visualize_node(self.root)


  def __traverse(self, node: DecisionNode, X: dict):
    """
    Recursively traverse the tree to make a prediction.

    Parameters
    ----------
    node : DecisionNode
        Current node in traversal.
    X : dict
        Single sample represented as a dictionary of feature-value pairs.

    Returns
    -------
    float
        Predicted value from the leaf node.
    """

    # If leaf node, return stored prediction
    if node.leaf_value:
      return node.leaf_value

    # Get the feature value for current split
    feature_value = X[node.feature]

    # Move to the corresponding child node
    next_node = node.branches[feature_value]

    # Continue traversal
    return self.__traverse(next_node, X)


  def predict(self, X: dict):
    """
    Predict the output for a single sample.

    Parameters
    ----------
    X : dict
        Feature dictionary for one data sample.

    Returns
    -------
    float
        Predicted value.
    """

    return self.__traverse(self.root, X)


# Fitting a dataset on the decision tree

An attempt to replicate decision tree from: <br/>
http://www.saedsayad.com/decision_tree_reg.htm

In [398]:
# Dummy data
data = {
    "Outlook": ["Rainy", "Rainy", "Overcast", "Sunny", "Sunny", "Sunny", "Overcast", "Rainy", "Rainy", "Sunny", "Rainy", "Overcast", "Overcast", "Sunny"],
    "Temp": ["Hot", "Hot", "Hot", "Mild", "Cool", "Cool", "Cool", "Mild", "Cool", "Mild", "Mild", "Mild", "Hot", "Mild"],
    "Humidity": ["High", "High", "High", "High", "Normal", "Normal", "Normal", "High", "Normal", "Normal", "Normal", "High", "Normal", "High"],
    "Windy": ["False", "True", "False", "False", "False", "True", "True", "False", "False", "False", "True", "True", "False", "True"],
    "Windy": ["False", "True", "False", "False", "False", "True", "True", "False", "False", "False", "True", "True", "False", "True"],
    "HoursPlayed": [25,30,46,45,52,23,43,35,38,46,48,52,44,30],
}

df = pd.DataFrame(data)

df

,Outlook,Temp,Humidity,Windy,HoursPlayed
0,Rainy,Hot,High,False,25
1,Rainy,Hot,High,True,30
2,Overcast,Hot,High,False,46
3,Sunny,Mild,High,False,45
4,Sunny,Cool,Normal,False,52
5,Sunny,Cool,Normal,True,23
6,Overcast,Cool,Normal,True,43
7,Rainy,Mild,High,False,35
8,Rainy,Cool,Normal,False,38
9,Sunny,Mild,Normal,False,46


In [399]:
# Fitting decision tree
tree = DecisionTree(max_depth=4, min_samples=3, spitting_threhold=0.1)
tree.fit(df.drop("HoursPlayed", axis=1), df["HoursPlayed"])

## Visualizing decision tree

In [400]:
tree.visualize()

[ Outlook ]
 -- Rainy --> [ Temp ]
	 -- Hot --> 27.5
	 -- Mild --> 41.5
	 -- Cool --> 38.0
 -- Overcast --> 46.25
 -- Sunny --> [ Windy ]
	 -- False --> 47.666666666666664
	 -- True --> 26.5


In [401]:
# Inferencing

In [402]:
# Dummy new data
new_data_dict = {"Outlook": "Sunny", "Temp": "Mild", "Humidity": "High", "Windy": "False"}
new_data_dict

{'Outlook': 'Sunny', 'Temp': 'Mild', 'Humidity': 'High', 'Windy': 'False'}

In [403]:
# Predicting
tree.predict(new_data_dict)

47.666666666666664